In [1]:
!pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms.functional as F
from PIL import Image



In [3]:
!unzip hand_data.zip

Archive:  hand_data.zip
   creating: hand_data/
   creating: hand_data/train/
   creating: hand_data/train/Hello/
  inflating: hand_data/train/Hello/1715432223741.jpg  
  inflating: hand_data/train/Hello/1715432223741_darker.jpg  
  inflating: hand_data/train/Hello/1715432223754.jpg  
  inflating: hand_data/train/Hello/1715432223754_darker.jpg  
  inflating: hand_data/train/Hello/1715432223754_lighter.jpg  
  inflating: hand_data/train/Hello/1715432223767.jpg  
  inflating: hand_data/train/Hello/1715432223767_darker.jpg  
  inflating: hand_data/train/Hello/1715432223767_lighter.jpg  
  inflating: hand_data/train/Hello/1715432223785.jpg  
  inflating: hand_data/train/Hello/1715432223785_darker.jpg  
  inflating: hand_data/train/Hello/1715432223785_lighter.jpg  
  inflating: hand_data/train/Hello/1715432223799.jpg  
  inflating: hand_data/train/Hello/1715432223799_darker.jpg  
  inflating: hand_data/train/Hello/1715432223799_lighter.jpg  
  inflating: hand_data/train/Hello/1715432223813.

In [4]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset
train_data = datasets.ImageFolder('hand_data/train', transform=transform)
val_data = datasets.ImageFolder('hand_data/val', transform=transform)
print(len(train_data), len(val_data))


# Create data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=4)

880 204


In [5]:
# Load pre-trained model
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
# Replace the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 88.4MB/s]


In [6]:
# Training loop
num_epochs = 45
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels).item()

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, num_epochs, val_loss/len(val_loader), 100 * correct / total))


Epoch [1/45], Loss: 0.3770, Accuracy: 85.29%
Epoch [2/45], Loss: 0.1958, Accuracy: 94.12%
Epoch [3/45], Loss: 0.1352, Accuracy: 95.59%
Epoch [4/45], Loss: 0.0456, Accuracy: 100.00%
Epoch [5/45], Loss: 0.0514, Accuracy: 99.51%
Epoch [6/45], Loss: 0.0751, Accuracy: 99.02%
Epoch [7/45], Loss: 0.0629, Accuracy: 98.53%
Epoch [8/45], Loss: 0.0428, Accuracy: 98.53%
Epoch [9/45], Loss: 0.0462, Accuracy: 99.02%
Epoch [10/45], Loss: 0.0610, Accuracy: 99.02%
Epoch [11/45], Loss: 0.0462, Accuracy: 99.02%
Epoch [12/45], Loss: 0.0570, Accuracy: 97.55%
Epoch [13/45], Loss: 0.0558, Accuracy: 98.04%
Epoch [14/45], Loss: 0.0379, Accuracy: 98.53%
Epoch [15/45], Loss: 0.0455, Accuracy: 98.04%
Epoch [16/45], Loss: 0.0534, Accuracy: 98.04%
Epoch [17/45], Loss: 0.0442, Accuracy: 99.02%
Epoch [18/45], Loss: 0.1861, Accuracy: 92.65%
Epoch [19/45], Loss: 0.0407, Accuracy: 98.53%
Epoch [20/45], Loss: 0.0499, Accuracy: 98.04%
Epoch [21/45], Loss: 0.0346, Accuracy: 98.53%
Epoch [22/45], Loss: 0.0350, Accuracy: 98.

In [7]:
torch.save(model.state_dict(), 'model_params.pth')